In [4]:
from urllib.parse import urlparse

In [5]:
parsed_url='https://www.kaggle.com/code/damiroan/comparison-of-each-model-lr-sgd-dt-rf-gb/comments'

In [6]:
urlparsed = urlparse(parsed_url)

In [7]:
urlparsed.path.strip('/').split('/')

['code', 'damiroan', 'comparison-of-each-model-lr-sgd-dt-rf-gb', 'comments']